In [5]:
%pip install pandas
%pip install pydmd
%pip install scikit-learn
%pip install matplotlib
%pip install seaborn
%pip install keras_tuner
%pip install keras


/bin/bash: /home/yurid/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/yurid/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/yurid/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/yurid/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/yurid/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.
/bin/bash: /home/yurid/miniconda3/envs/tf/lib/libtinfo.so.6: no version informat

In [1]:
import gc

from src.filler import dmd_filler, interpolate_filler, fill_df
from src.dataframe import read_data, windowing, split_data, show_error_metrics
from src.scaler import Scaler
from src.model import train_model, hp_search
from src.model import hp_search, build_model
from keras.callbacks import EarlyStopping
from matplotlib import pyplot
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM, InputLayer
from keras.callbacks import EarlyStopping
import keras

import numpy as np
from pydmd import DMD

ModuleNotFoundError: No module named 'src'

In [7]:
cities = {
    'barbacena': 'daily_barbacena',
    'juiz de fora': 'daily_juiz de fora',
    'muriae': 'daily_muriae',
    'sao joao del rei': 'daily_sao joao del rei',
    'vicosa': 'daily_vicosa'
}

fillers = {'dmd':dmd_filler, 'interpolate':interpolate_filler}

In [8]:
for city in cities:
    for filler in fillers:
        print(f'=== {city} = {filler} ===')

        name = f'{filler}_lstm_5_1_{city}'

        # read
        file = f'../data/{cities[city]}.csv'
        df = read_data(file)
        df = fill_df(df, fillers[filler])

        # scaler
        scaler = Scaler(df)
        df_scaled = scaler.get_dataframe_scaled()

        # windowing
        X, Y = windowing(df_scaled, 5, 1)
        train_x, train_y, test_x, test_y = split_data(X, Y, length=len(df_scaled), ratio=0.9)

        hp = hp_search(name,
                       train_x,
                       train_y,
                       max_trials=50,
                       executions_per_trial=10,
                       epochs=15,
                       validation_split=0.3,
                       verbose=1)

        model = build_model(hp)
        print(hp.values)


        # fit network
        model = train_model(model,
                            train_x=train_x,
                            train_y=train_y,
                            name_model=f'model_{name}',
                            batch_size=72,
                            verbose=1,
                            early_stopping_patience= 100,
                            cache_path='./cache')


        scores = model.evaluate(X, Y, verbose=0)
        print(scores)


        # predict
        predict = model.predict(test_x)
        real = scaler.scales['RADIATION'].inverse_transform(test_y.reshape(-1,1))
        predict = scaler.scales['RADIATION'].inverse_transform(predict)

        # show predict
        range_plot = 100
        pyplot.plot(real[0:range_plot], color='blue', label='real')
        pyplot.plot(predict[0:range_plot], color='red', label='incompleto')
        pyplot.show()

        show_error_metrics(real, predict)

        print('\n\n\n')
        gc.collect()

Trial 21 Complete [00h 08m 44s]
val_loss: 0.021206262148916723

Best val_loss So Far: 0.021160120144486427
Total elapsed time: 00h 08m 44s

Search: Running Trial #22

Value             |Best Value So Far |Hyperparameter
384               |288               |units
0.1               |0.25              |dropout
160               |416               |units_2
relu              |sigmoid           |activation
0.01              |0.01              |learning_rate
Epoch 1/15
145/145 [==============================] - 7s 31ms/step - loss: 0.2714 - val_loss: 0.2942
Epoch 2/15
145/145 [==============================] - 4s 25ms/step - loss: 0.2697 - val_loss: 0.2942
Epoch 3/15
145/145 [==============================] - 4s 24ms/step - loss: 0.2697 - val_loss: 0.2942
Epoch 4/15
145/145 [==============================] - 4s 24ms/step - loss: 0.2697 - val_loss: 0.2942
Epoch 5/15
145/145 [==============================] - 4s 28ms/step - loss: 0.2697 - val_loss: 0.2942
Epoch 6/15
145/145 [==================

Traceback (most recent call last):
  File "/home/yurid/miniconda3/envs/tf/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_50079/3729540775.py", line 20, in <module>
    hp = hp_search(name,
  File "/home/yurid/git/tcc/notebook/src/model.py", line 62, in hp_search
  File "/home/yurid/miniconda3/envs/tf/lib/python3.9/site-packages/keras_tuner/engine/base_tuner.py", line 230, in search
    self._try_run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/yurid/miniconda3/envs/tf/lib/python3.9/site-packages/keras_tuner/engine/base_tuner.py", line 270, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/yurid/miniconda3/envs/tf/lib/python3.9/site-packages/keras_tuner/engine/base_tuner.py", line 235, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/yurid/miniconda3